In [ ]:
# # ONLY IF REQUIRED, Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install navabilitysdk

In [ ]:
from navability.entities import *
from navability.services import *
from uuid import uuid4
import asyncio
import numpy as np

In [ ]:

navability_client = NavAbilityHttpsClient()
client = Client("guest@navability.io", "SDKpy_" + str(uuid4())[0:4], "Tutorial4_" + str(uuid4())[0:4])



In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
MapVizApp(context)


In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

In [ ]:

result_id = await addVariable(navability_client, client, Variable("l0", VariableType.Pose2.value))
result_id = await addVariable(navability_client, client, Variable("l1", VariableType.Pose2.value))
result_id = await addVariable(navability_client, client, Variable("l2", VariableType.Pose2.value))
result_id = await addVariable(navability_client, client, Variable("l3", VariableType.Pose2.value))
result_id = await addVariable(navability_client, client, Variable("l4_h1", VariableType.Pose2.value))
result_id = await addVariable(navability_client, client, Variable("l4_h2", VariableType.Pose2.value))


prior_distr= np.power(np.diag([0.1, 0.1, 0.01]),2)

f = Factor("l0f1", "PriorPose2", ["l0"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[0.,0, 0], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 
f = Factor("l1f1", "PriorPose2", ["l1"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20.,0, np.pi/2], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 
f = Factor("l2f1", "PriorPose2", ["l2"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20.,10, np.pi], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 
f = Factor("l3f1", "PriorPose2", ["l3"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[0.,10, -np.pi/2], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 

f = Factor("l4_h1f1", "PriorPose2", ["l4_h1"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20., 4, 0], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 
f = Factor("l4_h2f1", "PriorPose2", ["l4_h2"], FactorData(fnc=PriorPose2(Z=FullNormal(mu=[20., 4, np.pi], cov=prior_distr)).dump()))
result_id = await addFactor(navability_client, client, f) 



In [ ]:

result_id = await addVariable(navability_client, client, Variable("x0", VariableType.Pose2.value))


f = Factor("x0l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x0", "l0", "l1", "l2", "l3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[-2.,-2, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(navability_client, client, f) 


In [ ]:
solve_request = await solveSession(navability_client, client)


In [ ]:
result_id = await addVariable(navability_client, client, Variable("x1", VariableType.Pose2.value))

f = Factor("x0x1f1", 
           "Pose2Pose2", 
           ["x0", "x1"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[4.,0, np.pi/2], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump())
)
result_id = await addFactor(navability_client, client, f) 

f = Factor("x1l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x1", "l0", "l1", "l2", "l3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[-2.,-4, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(navability_client, client, f) 


In [ ]:
solve_request = await solveSession(navability_client, client)


In [ ]:
result_id = await addVariable(navability_client, client, Variable("x2", VariableType.Pose2.value))

f = Factor("x1x2f1", 
           "Pose2Pose2", 
           ["x1", "x2"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[16.,0, 0], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump())
)
result_id = await addFactor(navability_client, client, f) 

f = Factor("x2l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x2", "l0", "l1", "l2", "l3"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[2.,-4, np.pi/2], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
)
result_id = await addFactor(navability_client, client, f) 


In [ ]:
solve_request = await solveSession(navability_client, client)


In [ ]:
f = Factor("x2l4f1", 
           "Pose2Pose2", 
           ["x2", "l4_h1", "l4_h2"], 
           FactorData(fnc=PriorPose2(Z=FullNormal(mu=[2., 0, np.pi], cov=np.power(np.diag([0.5, 0.5, 0.05]),2))).dump(), 
                      multihypo=[1.0, 0.5, 0.5])
)
result_id = await addFactor(navability_client, client, f) 

In [ ]:
solve_request = await solveSession(navability_client, client)
